In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from urllib.request import urlopen
import datetime as dt
from datetime import datetime

# To Plot matplotlib figures inline on the notebook
%matplotlib inline

## Field Descriptions


C/A      = Control Area (A002)  
UNIT     = Remote Unit for a station (R051)  
SCP      = Subunit Channel Position represents an specific address for a device (02-00-00)  
STATION  = Represents the station name the device is located at  
LINENAME = Represents all train lines that can be boarded at this station
           Normally lines are represented by one character.  LINENAME 456NQR repersents train server for 4, 5, 6, N, Q, and R trains.  
DIVISION = Represents the Line originally the station belonged to BMT, IRT, or IND  
DATE     = Represents the date (MM-DD-YY)  
TIME     = Represents the time (hh:mm:ss) for a scheduled audit event  
DESc     = Represent the "REGULAR" scheduled audit event (Normally occurs every 4 hours)  
           1. Audits may occur more that 4 hours due to planning, or troubleshooting activities.  
           2. Additionally, there may be a "RECOVR AUD" entry: This refers to a missed audit that was recovered.  
ENTRIES  = The comulative entry register value for a device  
EXIST    = The cumulative exit register value for a device  

In [2]:
url1 = 'http://web.mta.info/developers/data/nyct/turnstile/turnstile_170225.txt'
df1 = pd.read_csv(url1)

url2 = 'http://web.mta.info/developers/data/nyct/turnstile/turnstile_170304.txt'
df2 = pd.read_csv(url2)

url3 = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_170311.txt"
df3 =pd.read_csv(url3)
url4 = 'http://web.mta.info/developers/data/nyct/turnstile/turnstile_170318.txt'
df4= pd.read_csv(url4)
url5 = 'http://web.mta.info/developers/data/nyct/turnstile/turnstile_170325.txt'
df5 = pd.read_csv(url5)

url6 = 'http://web.mta.info/developers/data/nyct/turnstile/turnstile_170401.txt'
df6=pd.read_csv(url6)
url7 = 'http://web.mta.info/developers/data/nyct/turnstile/turnstile_170408.txt'
df7 = pd.read_csv(url7)
url8 = 'http://web.mta.info/developers/data/nyct/turnstile/turnstile_170415.txt'
df8 = pd.read_csv(url8)
url9 = 'http://web.mta.info/developers/data/nyct/turnstile/turnstile_170422.txt'
df9 = pd.read_csv(url9)
url10 = 'http://web.mta.info/developers/data/nyct/turnstile/turnstile_170429.txt'
df10 = pd.read_csv(url10)
url11 = 'http://web.mta.info/developers/data/nyct/turnstile/turnstile_170506.txt'
df11 = pd.read_csv(url11)
url12 = 'http://web.mta.info/developers/data/nyct/turnstile/turnstile_170513.txt'
df12 = pd.read_csv(url12)
url13 = 'http://web.mta.info/developers/data/nyct/turnstile/turnstile_170520.txt'
df13= pd.read_csv(url13)
url14 = 'http://web.mta.info/developers/data/nyct/turnstile/turnstile_170527.txt'
df14= pd.read_csv(url14)
url15 = 'http://web.mta.info/developers/data/nyct/turnstile/turnstile_170603.txt'
df15 = pd.read_csv(url15)
url16 = 'http://web.mta.info/developers/data/nyct/turnstile/turnstile_170610.txt'
df16 = pd.read_csv(url16)
url17 = 'http://web.mta.info/developers/data/nyct/turnstile/turnstile_170617.txt'
df17 = pd.read_csv(url17)
url18 = 'http://web.mta.info/developers/data/nyct/turnstile/turnstile_170624.txt'
df18 = pd.read_csv(url18)
frames = [df1, df2, df3, df4, df5, df6, df7, df8, df9,\
          df10, df11, df12, df13,df14,df15,df16,df17, df18]

df = pd.concat(frames)
df.tail()
#SCP is turnstile

KeyboardInterrupt: 

In [ ]:
df.columns = [column.strip() for column in df.columns]

In [ ]:
df['DATETIME'] = df['DATE'] + ' ' + df['TIME']
df['DATETIME'] = df['DATETIME'].apply(lambda x: dt.datetime.strptime(x, '%m/%d/%Y %H:%M:%S'))

weekdays = ['Monday', 'Tuesday', 
            'Wednesday', 'Thursday', 
            'Friday', 'Saturday', 'Sunday']

df['Day'] = df['DATETIME'].apply(lambda x: weekdays[x.weekday()])
df['Hour'] = df['DATETIME'].apply(lambda x: x.hour)
df['Week_Num'] = df['DATETIME'].apply(lambda x: x.week)

In [ ]:
(df.groupby(["SCP","STATION", "DATETIME","UNIT","C/A"])
 .ENTRIES.count()
 .reset_index()
 .sort_values("ENTRIES", ascending=False)).head(50)

df.drop_duplicates(subset=["C/A", "UNIT", "SCP", "STATION", "DATETIME"],inplace=True)

In [ ]:
#df=df.drop(['DIVISION','LINENAME','DESC','C/A','UNIT','SCP'],axis=1,errors='ignore')
df.head()

In [ ]:
df[["PREV_DATE", "PREV_ENTRIES"]] = (df
                                       .groupby(["C/A", "UNIT", "SCP", "STATION"])["DATE", "ENTRIES"]
                                       .transform(lambda grp: grp.shift(1)))

In [ ]:
df.dropna(subset=["PREV_DATE"], axis=0, inplace=True)
#df=df[df["ENTRIES"] > df["PREV_ENTRIES"]]
df.head

In [ ]:
def get_daily_counts(row, max_counter):
    counter = row["ENTRIES"] - row["PREV_ENTRIES"]
    if counter < 0:
        # May be counter is reversed?
        counter = -counter
    if counter > max_counter:
        print(row["ENTRIES"], row["PREV_ENTRIES"],row['C/A'],row['UNIT'],
             row['SCP'],row['STATION'])
        # assumption is that 
        counter = min(row["ENTRIES"], row["PREV_ENTRIES"])
    if counter > max_counter:
        # Check it again to make sure we are not giving a counter that's too big
        return 0
    return counter

# If counter is > 1Million, then the counter might have been reset.  
# Just set it to zero as different counters have different cycle limits
df["DAILY_ENTRIES"] = turnstiles_daily.apply(get_daily_counts, axis=1, max_counter=1000000)

In [ ]:
#df.groupby(df['Day']).sum().head() #.plot(kind='barh');

In [ ]:
# Code for getting the distribution of the entries for day of the week
df['DATE_TIME'] = df['DATE'] + ' ' + df['TIME']
df["DATE_TIME"] = pd.to_datetime(turnstiles_df.DATE + " " +turnstiles_df.TIME, format="%m/%d/%Y %H:%M:%S")
weekdays = ['Monday', 'Tuesday', 
            'Wednesday', 'Thursday', 
            'Friday', 'Saturday', 'Sunday']

df['Day'] = df['DATETIME'].apply(lambda x: weekdays[x.weekday()])

df.groupby(['Day']).sort_values(ascending=False).plot(kind=barh)
len(df.groupby.)